In [38]:
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
import seaborn as sns

In [39]:
dataset = pd.read_csv('dataset.csv')
dataset

,edge_id,length,minute,speed,is_holiday,weather
0,246,1726,505,22.140185,1,1
1,316,1750,740,54.303369,0,0
2,387,1612,745,56.303877,0,1
3,303,1927,1335,23.641228,0,2
4,250,1901,915,41.503161,0,1
...,...,...,...,...,...,...
784219,221,1851,1015,27.068770,0,1
784220,227,1768,175,47.168667,0,1
784221,110,1699,1220,28.605635,0,1
784222,289,1750,1410,21.889339,0,1


In [40]:
dataset.describe()

,edge_id,length,minute,speed,is_holiday,weather
count,784224.000000,784224.000000,784224.000000,768540.000000,784224.000000,784224.000000
mean,194.876607,1742.886889,717.500000,42.649084,0.142857,0.450133
std,112.482005,144.971298,415.689953,12.845111,0.349927,0.589825
min,0.000000,1501.000000,0.000000,3.290549,0.000000,0.000000
25%,98.000000,1612.000000,358.750000,34.569791,0.000000,0.000000
50%,195.000000,1741.000000,717.500000,42.591930,0.000000,0.000000
75%,292.000000,1865.000000,1076.250000,50.520944,0.000000,1.000000
max,389.000000,2000.000000,1435.000000,137.670425,1.000000,2.000000


هم واحد کردن ستون ها
done✅

In [41]:
# length: meter
# speed: km/h -> should be turned to meter/second (cause we want estimate the weight of each edge with seconds)

dataset['speed'] = dataset['speed'] * 10 / 36
dataset

,edge_id,length,minute,speed,is_holiday,weather
0,246,1726,505,6.150051,1,1
1,316,1750,740,15.084269,0,0
2,387,1612,745,15.639966,0,1
3,303,1927,1335,6.567008,0,2
4,250,1901,915,11.528656,0,1
...,...,...,...,...,...,...
784219,221,1851,1015,7.519103,0,1
784220,227,1768,175,13.102407,0,1
784221,110,1699,1220,7.946010,0,1
784222,289,1750,1410,6.080372,0,1


حذف داده های پرت از سرعت
(باید پیش از جایگزینی مقادیر نن بیاید که روی پر کردن انها تاثیر داده نشوند)
done✅

In [42]:
# detecting outlayers using z-score and paying attention to weather and holiday
# and splitting our time in two(step1 and step2) based on how crowded the streets at that time are

In [43]:
# from midnight to 6 AM

step1 = pd.DataFrame(dataset[(0<dataset.minute) & (dataset.minute<360)])
step1

,edge_id,length,minute,speed,is_holiday,weather
15,298,1587,240,10.161582,0,1
16,110,1699,225,13.359588,0,1
17,338,1747,160,15.773128,0,0
25,191,1758,275,11.458147,0,1
27,61,1894,105,15.886921,0,0
...,...,...,...,...,...,...
784188,75,1627,50,18.272743,0,0
784192,245,1520,5,18.215388,0,0
784202,149,1629,155,14.799370,0,0
784216,221,1851,45,12.055211,0,1


In [44]:
df2 = step1.groupby(['weather', 'is_holiday', 'edge_id'])
dataset['speed_norm'] = df2['speed'].apply(lambda x: abs(x-x.mean())/x.std())
dataset.drop(dataset[dataset.speed_norm>3].index, inplace=True)
dataset

,edge_id,length,minute,speed,is_holiday,weather,speed_norm
0,246,1726,505,6.150051,1,1,NaN
1,316,1750,740,15.084269,0,0,NaN
2,387,1612,745,15.639966,0,1,NaN
3,303,1927,1335,6.567008,0,2,NaN
4,250,1901,915,11.528656,0,1,NaN
...,...,...,...,...,...,...,...
784219,221,1851,1015,7.519103,0,1,NaN
784220,227,1768,175,13.102407,0,1,0.321899
784221,110,1699,1220,7.946010,0,1,NaN
784222,289,1750,1410,6.080372,0,1,NaN


In [45]:
# from 6 AM to the end of the day

step2 = pd.DataFrame(dataset[(360<dataset.minute) & (dataset.minute<1440)])
df3 = step2.groupby(['weather', 'is_holiday', 'edge_id'])
dataset['speed_norm'] = df3['speed'].apply(lambda x: abs(x-x.mean())/x.std())
dataset.drop(dataset[dataset.speed_norm>3].index, inplace=True)
dataset

,edge_id,length,minute,speed,is_holiday,weather,speed_norm
0,246,1726,505,6.150051,1,1,0.993964
1,316,1750,740,15.084269,0,0,0.550006
2,387,1612,745,15.639966,0,1,2.512924
3,303,1927,1335,6.567008,0,2,0.851728
4,250,1901,915,11.528656,0,1,0.797688
...,...,...,...,...,...,...,...
784219,221,1851,1015,7.519103,0,1,0.859021
784220,227,1768,175,13.102407,0,1,NaN
784221,110,1699,1220,7.946010,0,1,0.763030
784222,289,1750,1410,6.080372,0,1,0.789596


In [46]:
dataset.drop('speed_norm', axis=1, inplace=True)
dataset

,edge_id,length,minute,speed,is_holiday,weather
0,246,1726,505,6.150051,1,1
1,316,1750,740,15.084269,0,0
2,387,1612,745,15.639966,0,1
3,303,1927,1335,6.567008,0,2
4,250,1901,915,11.528656,0,1
...,...,...,...,...,...,...
784219,221,1851,1015,7.519103,0,1
784220,227,1768,175,13.102407,0,1
784221,110,1699,1220,7.946010,0,1
784222,289,1750,1410,6.080372,0,1


جایگزین کردن مقادیر خالی در سرعت

In [47]:
# replacing NaN values with the mean speed of the rows that have the exact same values for other columns
# if they are not found, replacing it with the rows that have the exact same attributes but have different minutes

In [48]:
dataset.isnull().sum() # so only speed contains NaN values

edge_id           0
length            0
minute            0
speed         15684
is_holiday        0
weather           0
dtype: int64

In [49]:
# ADDITIONAL

speedNan = pd.DataFrame(dataset[dataset.isna().any(axis=1)])
speedNan.sort_values(by='edge_id', inplace=True)
speedNan

,edge_id,length,minute,speed,is_holiday,weather
98277,0,1618,1040,NaN,0,1
692577,0,1618,1175,NaN,0,1
424742,0,1618,545,NaN,1,1
275716,0,1618,525,NaN,0,1
110066,0,1618,620,NaN,1,0
...,...,...,...,...,...,...
308874,389,1920,90,NaN,1,0
419419,389,1920,800,NaN,1,1
482791,389,1920,1435,NaN,0,0
326162,389,1920,790,NaN,0,1


In [130]:
speedNotNan = dataset[dataset.speed.notna()].groupby(['weather', 'is_holiday', 'minute','edge_id'])['speed'].mean()

In [69]:
# ADDITIONAL

averageSpeed = pd.DataFrame(speedNotNan.values)
averageSpeed.columns = ['average']
averageSpeed

,average
0,17.443469
1,16.108191
2,15.506702
3,13.667092
4,16.380171
...,...
352067,8.535801
352068,4.516132
352069,10.199043
352070,8.632399


In [107]:
# turning the groupby object to a dataframe for merging it with dataset

s = speedNotNan.reset_index()
s.columns = ['weather', 'is_holiday', 'minute', 'edge_id', 'average_speed'] 
s

,weather,is_holiday,minute,edge_id,average_speed
0,0,0,0,0,17.443469
1,0,0,0,1,16.108191
2,0,0,0,2,15.506702
3,0,0,0,3,13.667092
4,0,0,0,4,16.380171
...,...,...,...,...,...
352067,2,1,1435,341,8.535801
352068,2,1,1435,342,4.516132
352069,2,1,1435,348,10.199043
352070,2,1,1435,369,8.632399


In [108]:
final = pd.merge(left=dataset, right=s, how='left', on=['weather', 'is_holiday', 'minute', 'edge_id'])
final

,edge_id,length,minute,speed,is_holiday,weather,average_speed
0,246,1726,505,6.150051,1,1,6.150051
1,316,1750,740,15.084269,0,0,14.821076
2,387,1612,745,15.639966,0,1,13.057822
3,303,1927,1335,6.567008,0,2,7.084402
4,250,1901,915,11.528656,0,1,10.567799
...,...,...,...,...,...,...,...
776221,221,1851,1015,7.519103,0,1,6.917949
776222,227,1768,175,13.102407,0,1,12.739193
776223,110,1699,1220,7.946010,0,1,6.545983
776224,289,1750,1410,6.080372,0,1,8.574255


In [131]:
# ADDITOINAL

final[(final.edge_id==83) & (final.length==1903) & (final.minute==775) & (final.is_holiday==0) & (final.weather==1)]

,edge_id,length,minute,speed,is_holiday,weather,average_speed
177837,83,1903,775,10.445484,0,1,11.364107
402482,83,1903,775,10.592577,0,1,11.364107
776225,83,1903,775,13.054260,0,1,11.364107


In [120]:
final[final.average_speed.isna()] # they are all the speeds that didn't have similar rows

,edge_id,length,minute,speed,is_holiday,weather,average_speed
295,210,2000,885,NaN,1,1,NaN
594,147,1706,465,NaN,1,0,NaN
687,266,1584,190,NaN,1,1,NaN
756,22,1841,1385,NaN,0,0,NaN
1137,36,1895,1340,NaN,1,1,NaN
...,...,...,...,...,...,...,...
774835,28,1533,1180,NaN,0,0,NaN
774937,271,1851,225,NaN,1,1,NaN
774968,343,1821,150,NaN,0,2,NaN
775679,227,1768,415,NaN,0,2,NaN


In [121]:
final[(final.edge_id==210) & (final.length==2000) & (final.minute==885) & (final.is_holiday==1) & (final.weather==1)]

# This Nan should be calculated with values with similar minutes and the exact same attributes

,edge_id,length,minute,speed,is_holiday,weather,average_speed
295,210,2000,885,NaN,1,1,NaN


In [125]:
# filling the NaN values for speed with average_speed

final['speed'].fillna(final['average_speed'], inplace=True)

In [128]:
final[final.isna().any(axis=1)]

,edge_id,length,minute,speed,is_holiday,weather,average_speed
295,210,2000,885,NaN,1,1,NaN
594,147,1706,465,NaN,1,0,NaN
687,266,1584,190,NaN,1,1,NaN
756,22,1841,1385,NaN,0,0,NaN
1137,36,1895,1340,NaN,1,1,NaN
...,...,...,...,...,...,...,...
774835,28,1533,1180,NaN,0,0,NaN
774937,271,1851,225,NaN,1,1,NaN
774968,343,1821,150,NaN,0,2,NaN
775679,227,1768,415,NaN,0,2,NaN


In [129]:
dataset = final
dataset

,edge_id,length,minute,speed,is_holiday,weather,average_speed
0,246,1726,505,6.150051,1,1,6.150051
1,316,1750,740,15.084269,0,0,14.821076
2,387,1612,745,15.639966,0,1,13.057822
3,303,1927,1335,6.567008,0,2,7.084402
4,250,1901,915,11.528656,0,1,10.567799
...,...,...,...,...,...,...,...
776221,221,1851,1015,7.519103,0,1,6.917949
776222,227,1768,175,13.102407,0,1,12.739193
776223,110,1699,1220,7.946010,0,1,6.545983
776224,289,1750,1410,6.080372,0,1,8.574255


In [ ]:
# for the ones who are still Nan and should be filled with similar values with different minute

In [133]:
# using the same method as before but not involving minute

speedNotNan2 = dataset[dataset.speed.notna()].groupby(['weather', 'is_holiday','edge_id'])['speed'].mean()
s2 = speedNotNan2.reset_index()
s2.columns = ['weather', 'is_holiday', 'edge_id', 'average_speed2'] 
s2

,weather,is_holiday,edge_id,average_speed2
0,0,0,0,11.871132
1,0,0,1,11.633924
2,0,0,2,13.657644
3,0,0,3,12.603853
4,0,0,4,13.893530
...,...,...,...,...
2329,2,1,385,5.289606
2330,2,1,386,8.317255
2331,2,1,387,6.067770
2332,2,1,388,9.002949


In [135]:
final2 = pd.merge(left=dataset, right=s2, how='left', on=['weather', 'is_holiday', 'edge_id'])
final2

,edge_id,length,minute,speed,is_holiday,weather,average_speed,average_speed2
0,246,1726,505,6.150051,1,1,6.150051,9.479563
1,316,1750,740,15.084269,0,0,14.821076,14.348551
2,387,1612,745,15.639966,0,1,13.057822,11.037967
3,303,1927,1335,6.567008,0,2,7.084402,5.613769
4,250,1901,915,11.528656,0,1,10.567799,10.599337
...,...,...,...,...,...,...,...,...
776221,221,1851,1015,7.519103,0,1,6.917949,10.569039
776222,227,1768,175,13.102407,0,1,12.739193,10.877486
776223,110,1699,1220,7.946010,0,1,6.545983,10.998724
776224,289,1750,1410,6.080372,0,1,8.574255,8.424670


In [137]:
# filling the NaN values for speed with average_speed

final2['speed'].fillna(final2['average_speed2'], inplace=True)
final2.isna().sum() # -> no speeds are NaN

edge_id              0
length               0
minute               0
speed                0
is_holiday           0
weather              0
average_speed     3507
average_speed2       0
dtype: int64

In [139]:
dataset = final2
# filling NaN values for average_speed with the speed in that row
dataset['average_speed'].fillna(dataset['speed'] ,inplace=True)
dataset.isna().sum()

edge_id           0
length            0
minute            0
speed             0
is_holiday        0
weather           0
average_speed     0
average_speed2    0
dtype: int64

In [140]:
# removing average_speed2 column
dataset.drop('average_speed2', axis=1, inplace=True)
dataset

,edge_id,length,minute,speed,is_holiday,weather,average_speed
0,246,1726,505,6.150051,1,1,6.150051
1,316,1750,740,15.084269,0,0,14.821076
2,387,1612,745,15.639966,0,1,13.057822
3,303,1927,1335,6.567008,0,2,7.084402
4,250,1901,915,11.528656,0,1,10.567799
...,...,...,...,...,...,...,...
776221,221,1851,1015,7.519103,0,1,6.917949
776222,227,1768,175,13.102407,0,1,12.739193
776223,110,1699,1220,7.946010,0,1,6.545983
776224,289,1750,1410,6.080372,0,1,8.574255


وزن دادن به یال ها

In [ ]:
dataset.groupby(['speed']).mean()

In [ ]:
a = dataset.groupby('edge_id')
a.head()

استخراج گره های مجاور و اطلاعات از گراف

In [ ]:
graph = nx.read_gpickle('network.gpickle')
plt.figure(3,figsize=(10,8)) 
nx.draw(graph, with_labels=True, node_size=500, font_weight='bold')
plt.show()

In [ ]:
from pyvis.network import Network

net=Network(notebook=True)
net.from_nx(graph)
net.show(name="example.html")

In [ ]:
# لیست برای مجاور ها

adj_dic = nx.to_dict_of_lists(graph)
adj_dic

تست کد

In [ ]:
testcases = pd.read_csv('test_cases.csv')
testcases